In [16]:
import pandas as pd
from datetime import datetime
from dateutil import relativedelta

In [24]:
# Считываем данные из файла
data=pd.read_csv("./Data/casc-resto.csv",sep=';')
data['RKDate']=pd.to_datetime(data['RKDate'])
fix_date=(datetime.strptime('2017-07-01', '%Y-%m-%d'))
# Группируем данные по идентификатору клиента и смотрим на данные


In [25]:

def agg(group):
    bought=0
    recency=10000
    spent = 0
    visitedTimes = 1
    nonCashSpent = 0
    visitedTimes1 = 1
    for index, row in group.iterrows():
        if row['RKDate'] <=  datetime.strptime('2017-12-31', '%Y-%m-%d')  and row['RKDate'] >= fix_date:
            bought=1
            
        if row['RKDate'] <= fix_date and (fix_date - row['RKDate']).days < recency:
            recency = (fix_date - row['RKDate']).days
            
#        datesBefore = list(filter(lambda d: d <= fix_date, row['RKDate']))
#        if len(datesBefore) == 0:
#            frequency=0
#        months  = 1 + (relativedelta.relativedelta(fix_date, min(datesBefore)) ).months
#        frequency=len(datesBefore) / months
        
#        if (row['RKDate'] <= fix_date):
#            visitedTimes+=1
#            spent+=row['SummAfterPointsUsage']
#            MonetaryValue= spent / visitedTimes
            
#        if (row['RKDate'] <= fix_date):
#            visitedTimes1+=1
#            nonCashSpent+= ( row['SummBasic'] - row['SummAfterPointsUsage'] )
#            someVar= nonCashSpent / visitedTimes1
            
    return bought,recency #,frequency,MonetaryValue,someVar
def func(group):
    bought, recency=agg(group)
    d = {}
    d['Bought_Or_Not'] = bought
    d['Recency'] = recency
#    d['Frequency'] = frequency
#    d['Monetary_Value'] = MonetaryValue
#    d['Some_Variable'] = someVar
   
    return pd.Series(d, index=['did_buy', 'Recency'])
#,'Frequency','Monetary_Value','Some_Variable'
groupedData = data.groupby("CustomerID").apply(func)
aggregatedData= groupedData.reset_index()

# функция для получения флага "покупал или не покупал" 
# функция для получения показателя Recency
# функция для получения показателя Frequency
# функция для получения показателя Monetary Value
# функция для вычисления собственного показателя


In [26]:
print(aggregatedData)

# Считываем данные из файла для слияния
dataToMergeWith =pd.read_csv("./Data/CASC_constant.csv",sep=';')

      CustomerID  did_buy  Recency
0        2728046      NaN    160.0
1        2728088      NaN    720.0
2        2728089      NaN     15.0
3        2728095      NaN    177.0
4        2728107      NaN    115.0
...          ...      ...      ...
9995     2913103      NaN    401.0
9996     2913106      NaN    126.0
9997     2913114      NaN     95.0
9998     2913123      NaN      8.0
9999     2913132      NaN     44.0

[10000 rows x 3 columns]
